In [ ]:
### Step 1: Install Required Libraries
!pip install tensorflow keras numpy pandas matplotlib opencv-python 

In [ ]:
!pip uninstall tensorflow -y
!pip cache purge
!pip install tensorflow --no-cache-dir

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageChops, ImageEnhance

### Dataset Path
DATASET_PATH = r"E:\Minor_project\FakeImageDetector\IMAGEdata"  # Change this path to your dataset folder

### Image Preprocessing
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation')

### Build Model
def build_model():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze pre-trained model layers
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

### Train & Save Model
MODEL_PATH = "deepfake_detector.h5"

if not os.path.exists(MODEL_PATH):
    model = build_model()
    model.fit(train_generator, validation_data=val_generator, epochs=5)
    model.save(MODEL_PATH)
else:
    model = load_model(MODEL_PATH)
    print("Model loaded successfully.")

### ELA Analysis
def error_level_analysis(image_path, quality=90):
    """ Perform Error Level Analysis (ELA) on an image. """
    original = Image.open(image_path).convert('RGB')
    
    # Save and reload at a lower quality to highlight differences
    temp_path = "temp_ela.jpg"
    original.save(temp_path, 'JPEG', quality=quality)
    compressed = Image.open(temp_path)
    
    # Compute the absolute difference
    ela_image = ImageChops.difference(original, compressed)
    extrema = ela_image.getextrema()
    max_diff = max([x[1] for x in extrema])
    scale = 255 / max_diff if max_diff > 0 else 1
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    # Convert to numpy array and return
    return np.array(ela_image), ela_image

### Detect Fake or Real Image with ELA Visualization
def detect_fake(image_path):
    image_name = os.path.basename(image_path)
    
    # Perform ELA
    ela_array, ela_image = error_level_analysis(image_path)
    
    # Display ELA result
    plt.figure(figsize=(10,5))
    plt.subplot(1, 2, 1)
    plt.imshow(load_img(image_path))
    plt.title("Original Image")
    plt.axis("off")
    
    plt.subplot(1, 2, 2)
    plt.imshow(ela_array)
    plt.title("ELA Image")
    plt.axis("off")
    
    plt.show()
    
    # Predict using the model
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)[0][0]
    result = "Real" if prediction > 0.5 else "Fake"
    print(f"{image_name}: {result}")

### Example Usage
detect_fake(r"E:\Minor_project\FakeImageDetector\Dataset\Fake\fake_0.jpg")  # Change to your test image path